In [2]:
import pandas as pd

path=r'D:\DATA_SCIENCE\BASES\Others\DataCamp\Financial_Data'

In [8]:
nasdaq = pd.read_csv(path+'\\nasdaq-listings.csv', na_values='n/a', parse_dates=['Last Update'])
# nasdaq = pd.read_excel(path+'\\listings.xlsx', sheetname=['nasdaq', 'nyse'], na_values='n/a')

In [9]:
nasdaq.head()

,Stock Symbol,Company Name,Last Sale,Market Capitalization,IPO Year,Sector,Industry,Last Update
0,AAPL,Apple Inc.,141.05,7.400000e+11,1980,Technology,Computer Manufacturing,2017-04-26
1,GOOGL,Alphabet Inc.,840.18,5.810000e+11,NAN,Technology,"Computer Software: Programming, Data Processing",2017-04-24
2,GOOG,Alphabet Inc.,823.56,5.690000e+11,2004,Technology,"Computer Software: Programming, Data Processing",2017-04-23
3,MSFT,Microsoft Corporation,64.95,5.020000e+11,1986,Technology,Computer Software: Prepackaged Software,2017-04-26
4,AMZN,"Amazon.com, Inc.",884.67,4.220000e+11,1997,Consumer Services,Catalog/Specialty Distribution,2017-04-24


In [ ]:
xls = pd.ExcelFile(path+'\\listings.xlsx')                              # Create pd.ExcelFile() object
sheets = xls.sheet_names                                                # Extract sheet names and store in sheets
print('Sheet names:', sheets)

nasdaq = pd.read_excel(xls, sheetname='nasdaq', na_values='n/a')
nyse   = pd.read_excel(xls, sheetname='nyse', na_values='n/a')
amex   = pd.read_excel(xls, sheetname='amex', na_values='n/a')

nasdaq['exchange'] = 'nasdaq'
nyse['exchange'] = 'nyse'
amex['exchange'] = 'amex'

listings = pd.concat([nasdaq, nyse, amex]).info()

In [ ]:
### Automated
xls = pd.ExcelFile(path+'\\listings.xlsx')                                 # Create pd.ExcelFile() object
exchanges = xls.sheet_names                                                # Extract sheet names and store in exchanges
listings=[]

for exchange in exchanges:
    listing = pd.read_excel(xls, sheetname=exchange, na_values='n/a')
    listing['Exchange'] = exchange
    listings.append(listing)

combined_listing = pd.concat(listings)

In [ ]:
### The DataReader: Access financial data online
from pandas_datareader.data import DataReader
from datetime import date

start = date(2015, 1, 1)
end = date(2016, 12, 31)
ticker = 'GOOG'
data_source = 'google'    # Google Finance

stock_data = DataReader(ticker, data_source, start, end)
stock_data.info()

In [ ]:
### Economic data from the Federal Reserve
from pandas_datareader.data import DataReader
from datetime import date

series_code = 'DGS10'   # 10-year Treasury Rate
data_source = 'fred'    # FED Economic Data Service
start = date(1962, 1, 1)

data = DataReader(series_code, data_source, start)

In [ ]:
### Economic data from the Federal Reserve
from pandas_datareader.data import DataReader
from datetime import date

start = date(2000, 1, 1)
series_code = 'DCOILWTICO'   # West Yexas Intermediate Oil Price
oil = DataReader(series_code, 'fred', start)

ticker = 'XOM'    # Exxon Mobile Corp
stock = DataReader(ticker, 'google', start)

data = pd.concat([stock[['Close']], oil], axis=1)  # Concat by columns
data.info()

data.columns = ['Exxon', 'Oil Price']
data.plot(); plt.show()

In [ ]:
# TWO SERIES
start = date(1950,1,1)
series = ['UNRATE', 'CIVPART']              # Define the series codes
econ_data = DataReader(series, 'fred', start)

econ_data.columns = ['Unemployment Rate', 'Participation Rate']
econ_data.plot(subplots=True, title='Labor Market');plt.show()

In [ ]:
# TOP
top5 = df['ColA'].nlargest(n=5)
top_3_companies = listings.loc[(listings.Sector=='Finance'), 'Market Capitalization'].nlargest(n=3)

In [ ]:
# manage a MultiIndex
nasdaq = pd.read_excel('listings.xlsx', sheetname='nasdaq', na_values='n/a')
nasdaq.set_index('Stock Symbol', inplace=True)
top_5 = nasdaq['Market Capitalization'].nlargest(n=5)
top_5.div(1000000)

tickers = top_5.index.tolist()    # Convert index to list
panel = DataReader(tickers, 'google', start=date(2015,1,1))
data = panel.to_frame()               # df with MultiIndex
data.info()

unstacked = data['Close'].unstack()

In [ ]:
### GROUP BY
nyse.info()
nyse['market_cap_m'] = nyse['Market Capitalization'].div(1e6)                          # Create market_cap_m
nyse = nyse.drop('Market Capitalization', axis=1)                                      # Drop market cap column
mcap_by_sector = nyse.groupby('Sector')                                                # Group nyse by sector
median_mcap_by_sector = mcap_by_sector.market_cap_m.median()                           # Calculate median

# Plot and show as horizontal bar chart
median_mcap_by_sector.plot(kind='barh', title='NYSE - Median Market Capitalization')
plt.xlabel('USD mn')
plt.show()

In [ ]:
### Company value by exchange and sector

by_sector_exchange = listings.groupby(['Sector', 'Exchange'])            # Group listings by Sector and Exchange
mcap_by_sector_exchange = by_sector_exchange.market_cap_m.median()       # Calculate the median market cap
print(mcap_by_sector_exchange.head())                                    # Display the head of the result

mcap_unstacked = mcap_by_sector_exchange.unstack()                       # Unstack mcap_by_sector_exchange

mcap_unstacked.plot(kind='bar', title='Median Market Capitalization by Exchange')    # Plot as a bar chart
plt.xlabel('USD mn')
plt.show()

In [ ]:
### Calculate several metrics by sector and exchange

listings['market_cap_m'] = listings['Market Capitalization'].div(1e6)
by_sector_exchange = listings.groupby(['Sector', 'Exchange'])

bse_mcm = by_sector_exchange['market_cap_m']
summary = bse_mcm.agg({'Average': 'mean', 'Median': 'median', 'Standard Deviation': 'std'})
print(summary)


In [ ]:
import seaborn as sns

listings = listings[(listings['IPO Year'] > 2000) & (listings.Exchange != 'amex')]  # Exclude IPOs > 2000 and from the 'amex'
listings['IPO Year'] = listings['IPO Year'].astype(int)                             # Convert IPO Year to integer
listings['market_cap_m'] = listings['Market Capitalization'].div(1e6)               # Create market_cap_m
listings = listings[listings.market_cap_m < listings.market_cap_m.quantile(.95)]    # Exclude outliers
sns.pointplot(x=market_cap_m, y='market_cap_m', hue='Exchange', data=listings)      # Create the pointplot
plt.xticks(rotation=45)                                                             # Rotate xticks
plt.show()